In [ ]:
!pip install --quiet google-api-python-client google-auth pandas

In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
import pandas as pd

drive = build('drive', 'v3')
sheets = build('sheets', 'v4')

FOLDER_ID = 'folder with tables'
EMAIL_SHEET_ID = 'table with e-mail (from A to A)'

In [ ]:
result = sheets.spreadsheets().values().get(
    spreadsheetId=EMAIL_SHEET_ID,
    range='Sheet1!A:A'
).execute()

expected_emails = {row[0].strip().lower() for row in result.get('values', []) if row}

In [ ]:
files = drive.files().list(
    q=f"'{FOLDER_ID}' in parents and mimeType='application/vnd.google-apps.spreadsheet'",
    fields="files(id, name)"
).execute().get('files', [])

print(f"Найдено таблиц: {len(files)}")

In [ ]:
report = []

for file in files:
    perms = drive.permissions().list(
        fileId=file['id'],
        fields="permissions(emailAddress, role)"
    ).execute()

    actual_emails = {p['emailAddress'].lower() for p in perms.get('permissions', []) if 'emailAddress' in p}

    for email in expected_emails:
        status = '✅ OK' if email in actual_emails else '❌ Missing'
        report.append({'File': file['name'], 'Email': email, 'Status': status})

    for email in actual_emails - expected_emails:
        report.append({'File': file['name'], 'Email': email, 'Status': '⚠️ Extra'})

In [ ]:
df = pd.DataFrame(report)
df

In [ ]:
# ====== Удаляем лишние доступы ======
from googleapiclient.errors import HttpError

for _, r in df.iterrows():
    if r.Status != '⚠️ Extra':
        continue

    file_id = None
    # ищем id файла по имени (можно кэшировать, если много)
    for f in files:
        if f['name'] == r.File:
            file_id = f['id']
            break
    if not file_id:
        continue

    # получаем список permission-ов
    perms = drive.permissions().list(fileId=file_id,
                                     fields='permissions(id,emailAddress)').execute()
    for p in perms.get('permissions', []):
        if p.get('emailAddress', '').lower() == r.Email.lower():
            try:
                drive.permissions().delete(fileId=file_id,
                                          permissionId=p['id']).execute()
                print(f"❌ Доступ закрыт: {r.Email} → {r.File}")
            except HttpError as e:
                print(f"Не удалось удалить {r.Email} ({r.File}): {e}")

In [ ]:
# ====== Выдаём editor-доступ тем, кого не хватало ======
from googleapiclient.errors import HttpError

for _, r in df.iterrows():
    if r.Status != '❌ Missing':
        continue

    # находим fileId по имени
    file_id = next((f['id'] for f in files if f['name'] == r.File), None)
    if not file_id:
        print(f'Файл не найден: {r.File}')
        continue

    try:
        drive.permissions().create(
            fileId=file_id,
            body={
                'type': 'user',
                'role': 'writer',   # writer = editor в UI
                'emailAddress': r.Email
            },
            sendNotificationEmail=False   # True — если нужно письмо
        ).execute()
        print(f'✅ Editor доступ выдан: {r.Email} → {r.File}')
    except HttpError as e:
        print(f'Не удалось выдать доступ {r.Email} ({r.File}): {e}')

In [ ]:
# ==== создаём новую таблицу и заливаем туда df ====
sheet_title = 'Access-Audit-' + pd.Timestamp.now().strftime('%Y-%m-%d')
spreadsheet = sheets.spreadsheets().create(body={'properties': {'title': sheet_title}}).execute()
new_id = spreadsheet['spreadsheetId']

# заливаем данные
sheets.spreadsheets().values().update(
    spreadsheetId=new_id,
    range='A1',
    valueInputOption='RAW',
    body={'values': [df.columns.values.tolist()] + df.values.tolist()}
).execute()

print('Готово! Ссылка на таблицу:')
print(f'https://docs.google.com/spreadsheets/d/{new_id}')